### Quickstart


In [39]:

from immuneML.data_model.dataset.ReceptorDataset import ReceptorDataset
from immuneML.data_model.dataset.RepertoireDataset import RepertoireDataset
from immuneML.data_model.dataset.SequenceDataset import SequenceDataset
from immuneML.environment.EnvironmentSettings import EnvironmentSettings
from immuneML.simulation.dataset_generation.RandomDatasetGenerator import RandomDatasetGenerator

In [40]:
from immuneML.caching.CacheType import CacheType
from immuneML.data_model.receptor.receptor_sequence.ReceptorSequence import ReceptorSequence
from immuneML.data_model.repertoire.Repertoire import Repertoire
from immuneML.environment.Constants import Constants
from immuneML.environment.EnvironmentSettings import EnvironmentSettings
from immuneML.simulation.implants.Motif import Motif
from immuneML.simulation.implants.Signal import Signal
from immuneML.simulation.motif_instantiation_strategy.GappedKmerInstantiation import GappedKmerInstantiation
from immuneML.simulation.sequence_implanting.GappedMotifImplanting import GappedMotifImplanting
from immuneML.simulation.signal_implanting_strategy.HealthySequenceImplanting import HealthySequenceImplanting
from immuneML.simulation.signal_implanting_strategy.ImplantingComputation import ImplantingComputation
from immuneML.util.PathBuilder import PathBuilder


In [41]:
path = EnvironmentSettings.tmp_test_path / \
    "random_repertoire_dataset_generation/"


In [42]:
path

WindowsPath('C:/Users/Karthik/.conda/envs/uni/lib/site-packages/test/tmp/random_repertoire_dataset_generation')

In [43]:
dataset = RandomDatasetGenerator.generate_repertoire_dataset(repertoire_count=100,
                                                             sequence_count_probabilities={
                                                                 100: 0.5, 120: 0.5},
                                                             sequence_length_probabilities={12: 0.33, 14: 0.33 , 15: 0.33}, labels={}, path=path)


In [16]:
# implanting = HealthySequenceImplanting(
#     GappedMotifImplanting(), implanting_computation=ImplantingComputation.ROUND)


In [17]:
# signal = Signal("sig1", [Motif(
#     "motif1", GappedKmerInstantiation(), "AA")], implanting)


In [ ]:
# signal

In [ ]:
# simulation = Simulation(implanting)

In [ ]:
type(dataset)

In [44]:
from immuneML.workflows.steps.SignalImplanter import SignalImplanter


In [45]:
from immuneML.simulation.SimulationState import SimulationState


In [20]:
# state = SimulationState(signal , simulation , dataset , name = "test"  , result_path= path)

NameError: name 'simulation' is not defined

In [21]:
# state

NameError: name 'state' is not defined

In [22]:
# dataset = SignalImplanter.run(state)

NameError: name 'state' is not defined

In [46]:
import logging
import os
import shutil
import sys
import warnings
from pathlib import Path

import yaml

from immuneML.app.ImmuneMLApp import ImmuneMLApp
from immuneML.environment.EnvironmentSettings import EnvironmentSettings
from immuneML.util.PathBuilder import PathBuilder


In [180]:
specs = {
          "definitions": {
               "datasets": {
                    "my_synthetic_dataset": {"format": "RandomRepertoireDataset", "params": {"labels": {}}}
                },
               "motifs": {"my_motif": {"seed": "AA", "instantiation": "GappedKmer"}},
                "signals": {"my_signal": {"motifs": ["my_motif"], "implanting": "HealthySequence"}},
                "simulations": {"my_simulation": {"my_implantng": {"signals": ["my_signal"], "dataset_implanting_rate": 0.5,
                                                                   "repertoire_implanting_rate": 0.1}}}
               },
           "instructions": {"simulation_instruction": {"type": "Simulation", "dataset": "my_synthetic_dataset", "simulation": "my_simulation",
                                                       "export_formats": ["AIRR"]}}
          }
specs_file = path / "simulation_specs.yaml"
with specs_file.open("w") as file:
    yaml.dump(specs, file)
app = ImmuneMLApp(specs_file, path / "result")
app.run()
print("immuneML quickstart: finished generating a synthetic dataset.")


2022-01-10 15:11:42.979304: Setting temporary cache path to ..\..\..\..\..\.conda\envs\uni\lib\site-packages\test\tmp\random_repertoire_dataset_generation\result\cache
2022-01-10 15:11:42.979304: ImmuneML: parsing the specification...

2022-01-10 15:11:43.952998: Full specification is available at ..\..\..\..\..\.conda\envs\uni\lib\site-packages\test\tmp\random_repertoire_dataset_generation\result\full_simulation_specs.yaml.

2022-01-10 15:11:43.953643: ImmuneML: starting the analysis...

2022-01-10 15:11:43.954170: Instruction 1/1 has started.


2022-01-10 15:11:47.374348: Instruction 1/1 has finished.
2022-01-10 15:11:47.386085: Generating HTML reports...
2022-01-10 15:11:51.251251: HTML reports are generated.
2022-01-10 15:11:51.252511: ImmuneML: finished analysis.

immuneML quickstart: finished generating a synthetic dataset.


In [ ]:
dataset

In [47]:
from immuneML.simulation.Implanting import Implanting
from immuneML.simulation.Simulation import Simulation


In [48]:
signal = Signal("sig1", [Motif(
    "motif1", GappedKmerInstantiation(), "AA")],   implanting_strategy=HealthySequenceImplanting(GappedMotifImplanting(), implanting_computation=ImplantingComputation.ROUND))


In [176]:
simulation = Simulation([Implanting(dataset_implanting_rate=0.5,
                        repertoire_implanting_rate=0.1, signals=[signal], name="i1")])


In [177]:
input_params = SimulationState(dataset=dataset, result_path=path,
                               simulation=simulation, signals=[signal], formats=["AIRR"])


In [179]:
len(new_dataset.get_example_ids())


100

In [178]:

new_dataset = SignalImplanter.run(input_params)


In [52]:
path

WindowsPath('C:/Users/Karthik/.conda/envs/uni/lib/site-packages/test/tmp/random_repertoire_dataset_generation')

In [53]:
import datetime
from collections import Counter
from pathlib import Path

import pandas as pd

from immuneML.IO.ml_method.MLExporter import MLExporter
from immuneML.environment.LabelConfiguration import LabelConfiguration
from immuneML.hyperparameter_optimization.config.SplitConfig import SplitConfig
from immuneML.hyperparameter_optimization.config.SplitType import SplitType
from immuneML.hyperparameter_optimization.core.HPAssessment import HPAssessment
from immuneML.hyperparameter_optimization.core.HPUtil import HPUtil
from immuneML.hyperparameter_optimization.states.TrainMLModelState import TrainMLModelState
from immuneML.hyperparameter_optimization.strategy.HPOptimizationStrategy import HPOptimizationStrategy
from immuneML.ml_metrics.Metric import Metric
from immuneML.reports.train_ml_model_reports.TrainMLModelReport import TrainMLModelReport
from immuneML.util.ReflectionHandler import ReflectionHandler
from immuneML.workflows.instructions.Instruction import Instruction
from immuneML.workflows.instructions.MLProcess import MLProcess
from scripts.specification_util import update_docs_per_mapping


In [54]:
import os
import shutil
from unittest import TestCase

import numpy as np

from immuneML.caching.CacheType import CacheType
from immuneML.data_model.dataset.RepertoireDataset import RepertoireDataset
from immuneML.data_model.encoded_data.EncodedData import EncodedData
from immuneML.environment.Constants import Constants
from immuneML.environment.EnvironmentSettings import EnvironmentSettings
from immuneML.ml_methods.LogisticRegression import LogisticRegression
from immuneML.workflows.steps.MLMethodTrainer import MLMethodTrainer
from immuneML.workflows.steps.MLMethodTrainerParams import MLMethodTrainerParams


In [55]:
from immuneML.encodings.kmer_frequency.KmerFrequencyEncoder import KmerFrequencyEncoder


In [ ]:
# from immuneML.encodings.kmer_frequency.KmerFreqRepertoireEncoder import KmerFreqRepertoireEncoder

In [68]:
ml_method = LogisticRegression({"C" : 0.1,
                               "penalty": "l1",
                               "max_iter": 200})

dataset = new_dataset

encoder = KmerFrequencyEncoder


In [326]:
from immuneML.hyperparameter_optimization.HPSetting import HPSetting
from immuneML.encodings.kmer_frequency.KmerFreqRepertoireEncoder import KmerFreqRepertoireEncoder
from immuneML.encodings.kmer_frequency.KmerFrequencyEncoder import KmerFrequencyEncoder

from immuneML.encodings.kmer_frequency.ReadsType import ReadsType
from immuneML.encodings.kmer_frequency.sequence_encoding.SequenceEncodingType import SequenceEncodingType
from immuneML.analysis.data_manipulation.NormalizationType import NormalizationType
Hpsetting1 = HPSetting(encoder=KmerFrequencyEncoder, encoder_params={"k":2, "reads": ReadsType.UNIQUE, "sequence_encoding": SequenceEncodingType.GAPPED_KMER, "normalization_type" : NormalizationType.RELATIVE_FREQUENCY}, ml_method=LogisticRegression({"C": 0.1, "penalty": "l1", "max_iter": 200}), ml_params={}, preproc_sequence=[], encoder_name="e1", ml_method_name="simpleLR")


In [327]:
Hpsetting2 = HPSetting(encoder=KmerFrequencyEncoder, encoder_params={
                       "k": 2, "reads": ReadsType.UNIQUE, "sequence_encoding": SequenceEncodingType.GAPPED_KMER, "normalization_type": NormalizationType.RELATIVE_FREQUENCY}, ml_method=LogisticRegression({"C": 0.1, "penalty": "l1", "max_iter": 200}), ml_params={}, preproc_sequence=[], encoder_name="e2", ml_method_name="simpleLR")


In [110]:
from immuneML.hyperparameter_optimization.core.HPUtil import HPUtil

In [64]:
from immuneML.hyperparameter_optimization.strategy.GridSearch import GridSearch

In [102]:
HPStrategy = GridSearch(hp_settings=[Hpsetting1,Hpsetting2])

In [118]:
from immuneML.hyperparameter_optimization.config.ReportConfig import ReportConfig


In [127]:
datasplits = {
    "SequenceLengthDistribution": {
        "batch_size": 3
    }
}


In [128]:
type(datasplits)

dict

In [129]:
models1= {"coef": "Coefficients"}


In [130]:
type(models1)

dict

In [183]:
reportconfig1 = ReportConfig(
    data_splits= datasplits, models= models1)


In [184]:
reportconfig2 = ReportConfig(
    data_splits= datasplits)


In [185]:
from immuneML.hyperparameter_optimization.config.SplitConfig import SplitConfig

In [193]:
split_config_assesment = SplitConfig(split_strategy=SplitType.RANDOM,split_count= 1 , training_percentage= 0.7 , reports= reportconfig1 )

In [194]:
split_config_selection = SplitConfig(
    split_strategy=SplitType.RANDOM, split_count=1, training_percentage=.07, reports=reportconfig2)


In [136]:
from immuneML.environment.LabelConfiguration import LabelConfiguration

In [138]:
from immuneML.environment.Label import Label

In [140]:
label = LabelConfiguration(labels = [Label("my_signal")])

In [141]:
from immuneML.ml_metrics.Metric import Metric

In [265]:
metrics = Metric.ACCURACY

In [266]:
isinstance(metrics, Metric)

True

In [267]:
type(metrics)

<enum 'Metric'>

In [268]:
optimisatization_metric = Metric.BALANCED_ACCURACY

In [260]:
type(optimisatization_metric)

<enum 'Metric'>

In [144]:
from immuneML.reports.train_ml_model_reports.MLSettingsPerformance import MLSettingsPerformance



reports = [MLSettingsPerformance]

In [161]:
number_of_processes = 3

refit_optimal_model = True


In [146]:
from immuneML.workflows.instructions.TrainMLModelInstruction import TrainMLModelInstruction

In [147]:
state = TrainMLModelInstructions()

NameError: name 'TrainMLModelInstructions' is not defined

# HP_split

In [199]:
train_val_datasets , test_datasets  = HPUtil.split_data(new_dataset , split_config_assesment , path , label )

In [203]:
type(new_dataset)

immuneML.data_model.dataset.RepertoireDataset.RepertoireDataset

In [200]:
nsplits = len(train_val_datasets)

In [202]:
nsplits

1

In [153]:
train_val_datasets

In [152]:
test_datasets

In [155]:
dataset

In [158]:
new_dataset.get_data()

In [173]:
dataset

In [164]:
state = TrainMLModelState(dataset=dataset, hp_strategy=HPStrategy, hp_settings=[Hpsetting1, Hpsetting2], assessment=split_config_assesment, selection=split_config_selection, metrics = metrics, optimization_metric = optimisatization_metric, label_configuration = label, path = path, refit_optimal_model= refit_optimal_model, number_of_processes= number_of_processes, reports={"hrep": MLSettingsPerformance})


In [165]:
state.name

In [166]:
state.path

WindowsPath('C:/Users/Karthik/.conda/envs/uni/lib/site-packages/test/tmp/random_repertoire_dataset_generation')

In [167]:
state = HPAssessment._create_root_path(state)

In [ ]:
for index in range(n)

In [228]:
train_val_datasets, test_datasets = HPUtil.split_data(
    state.dataset, state.assessment, state.path, state.label_configuration)
n_splits = len(train_val_datasets)


In [229]:
nsplits

1

In [230]:
dataset

In [171]:
type(dataset)

immuneML.data_model.dataset.RepertoireDataset.RepertoireDataset

In [231]:
for index in range(n_splits):
            state = HPAssessment.run_assessment_split(
                state, train_val_datasets[index], test_datasets[index], index, n_splits)


2022-01-10 15:58:14.158784: Training ML model: running outer CV loop: started split 1/1.

2022-01-10 15:58:14.174873: Hyperparameter optimization: running the inner loop of nested CV: selection for label my_signal (label 1 / 1).



TypeError: 'Metric' object is not iterable

In [192]:
dataset.get_example_count()

100

In [205]:
new_dataset.get_example_ids()

['4df51630949e4ac5a5bfdd7df177f3cc',
 'ab7117b15aab418aac66d0dba2b99c3f',
 '24bf074070d74f83bee847b89f8d16d8',
 '88d0ce7c1e824a7397ed715ab8d3d7fc',
 'd2d48977b5564453b235401a1dcf4889',
 '32aaa4e98042446b81cea0cbd216ca8b',
 'd3117bd1c1d74740a83eeae008cf44bf',
 'a7386e1a6ea44be5a946fe0264ddceb1',
 '382533976a3d4296957d7df113eddc81',
 '31473e71cc404de6b381cb0791fe2dc6',
 'fa2798a74cfd4be081382b6cb98debb2',
 '4fc5b01637894bc9ae10955b0478ced0',
 'ddb85fc4e2d44de0ac10542be2859ab6',
 'a6a2c67aa2d54674ac91a6a7f41df062',
 '53e523765d784c09934023463fbdadb5',
 '0e7e24d4028446b6bfc9747fea41d189',
 '5007d8466e61475f80e162a17c98e7b3',
 '74dc7033fbf7483484ec38144190d420',
 'ed6a157e361c4eca9d698c0d196bb616',
 '4f2e18ecbbad47ea8e9132493df5aa72',
 '8687799db6a8450aa7e303820dbf51ba',
 '2c00a3d91c7e4ba49675f508bffec4a6',
 'e7fd1a9e09514c1e8c69776e2e3225a6',
 'db7773ccc8ce4dc796f6ab92a51dc4df',
 '1c68132a41ff442fb7cbf8c5be0886fa',
 'e45584eec7e34b168e7bd9895b6479a8',
 'd72909c262e2441ea9ae3012fceba415',
 

In [208]:
new_dataset.get

WindowsPath('C:/Users/Karthik/.conda/envs/uni/lib/site-packages/test/tmp/random_repertoire_dataset_generation/metadata.csv')

In [214]:
import random

In [218]:
training_percentage = 0.7

In [219]:

train_count = int(dataset.get_example_count() * training_percentage)
train_datasets, test_datasets = [], []


In [220]:
indices = list(range(dataset.get_example_count()))
random.shuffle(indices)
train_index = indices[:train_count]
test_index = indices[train_count:]


In [222]:
len(train_index)

70

In [210]:
indices

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [225]:
from immuneML.workflows.steps.data_splitter.Util import Util
from immuneML.data_model.dataset.Dataset import Dataset

In [226]:
for i in range(1):

    indices = list(range(dataset.get_example_count()))
    random.shuffle(indices)
    train_index = indices[:train_count]
    test_index = indices[train_count:]
    train_dataset = Util.make_dataset(dataset, train_index, , i, Dataset.TRAIN)
    train_datasets.append(train_dataset)

    


TypeError: 'NoneType' object is not subscriptable

In [227]:
new_dataset.__class__.__name__

'RepertoireDataset'

In [ ]:
data

In [ ]:
input_params = Data

In [232]:

from immuneML.IO.dataset_export.AIRRExporter import AIRRExporter


In [233]:
path_exported = path / "exported"


In [237]:
path_exported

WindowsPath('C:/Users/Karthik/.conda/envs/uni/lib/site-packages/test/tmp/random_repertoire_dataset_generation/exported')

In [238]:
exported_dataset = AIRRExporter.export(new_dataset , path_exported)

In [240]:
exported_dataset


NoneType

In [236]:
type(exported_dataset)

NoneType

In [241]:
from immuneML.IO.dataset_import.AIRRImport import AIRRImport


In [242]:
column_mapping = {
    "junction": "sequences",
    "junction_aa": "sequence_aas",
    "v_call": "v_alleles",
    "j_call": "j_alleles",
    "locus": "chains",
    "duplicate_count": "counts",
    "sequence_id": "sequence_identifiers"
}


In [243]:
params = {"is_repertoire": True, "result_path": path_exported / "results", "path": path_exported, "metadata_file": path_exported / "metadata.csv",
          "import_out_of_frame": False, "import_with_stop_codon": False, "import_illegal_characters": False,
          "import_productive": True, "region_type": "IMGT_CDR3", "import_empty_nt_sequences": True, "import_empty_aa_sequences": False,
          "column_mapping": column_mapping,
          "separator": "\t"}


In [244]:
dataset = AIRRImport.import_dataset(params, "airr_repertoire_dataset")


In [245]:
train_val_datasets, test_datasets = HPUtil.split_data(
    dataset, split_config_assesment, path_exported / "train", label)


1

In [328]:
import datetime
from collections import Counter
from pathlib import Path

import pandas as pd

from immuneML.IO.ml_method.MLExporter import MLExporter
from immuneML.environment.LabelConfiguration import LabelConfiguration
from immuneML.hyperparameter_optimization.config.SplitConfig import SplitConfig
from immuneML.hyperparameter_optimization.config.SplitType import SplitType
from immuneML.hyperparameter_optimization.core.HPAssessment import HPAssessment
from immuneML.hyperparameter_optimization.core.HPUtil import HPUtil
from immuneML.hyperparameter_optimization.states.TrainMLModelState import TrainMLModelState
from immuneML.hyperparameter_optimization.strategy.HPOptimizationStrategy import HPOptimizationStrategy
from immuneML.ml_metrics.Metric import Metric
from immuneML.reports.train_ml_model_reports.TrainMLModelReport import TrainMLModelReport
from immuneML.util.ReflectionHandler import ReflectionHandler
from immuneML.workflows.instructions.Instruction import Instruction
from immuneML.workflows.instructions.MLProcess import MLProcess


In [329]:
state = TrainMLModelState(dataset=dataset, hp_strategy=HPStrategy, hp_settings=[Hpsetting1, Hpsetting2], assessment=split_config_assesment, selection=split_config_selection, metrics=set([metrics]),
                          optimization_metric=optimisatization_metric, label_configuration=label, path=path, refit_optimal_model=refit_optimal_model, number_of_processes=number_of_processes, reports={"hrep": MLSettingsPerformance})


In [330]:
type(metrics)

<enum 'Metric'>

In [331]:
state = HPAssessment.run_assessment(state)


2022-01-10 17:31:39.194914: Training ML model: running outer CV loop: started split 1/1.

2022-01-10 17:31:39.199833: Hyperparameter optimization: running the inner loop of nested CV: selection for label my_signal (label 1 / 1).

2022-01-10 17:31:39.201830: Evaluating hyperparameter setting: e1_simpleLR...
2022-01-10 17:31:39.202855: Encoding started...


TypeError: encode() missing 1 required positional argument: 'params'

In [271]:
state.metrics

<Metric.ACCURACY: 'accuracy_score'>

In [332]:
Hpsetting1.encoder_params

{'k': 2,
 'reads': <ReadsType.UNIQUE: 'unique'>,
 'sequence_encoding': <SequenceEncodingType.GAPPED_KMER: 'GappedKmerSequenceEncoder'>,
 'normalization_type': <NormalizationType.RELATIVE_FREQUENCY: 'l1'>}

In [334]:
Hpsetting1.encoder_params

{'k': 2,
 'reads': <ReadsType.UNIQUE: 'unique'>,
 'sequence_encoding': <SequenceEncodingType.GAPPED_KMER: 'GappedKmerSequenceEncoder'>,
 'normalization_type': <NormalizationType.RELATIVE_FREQUENCY: 'l1'>}

In [335]:
Hpsetting1.encoder_params

{'k': 2,
 'reads': <ReadsType.UNIQUE: 'unique'>,
 'sequence_encoding': <SequenceEncodingType.GAPPED_KMER: 'GappedKmerSequenceEncoder'>,
 'normalization_type': <NormalizationType.RELATIVE_FREQUENCY: 'l1'>}

In [337]:
Hpsetting1.encoder

immuneML.encodings.kmer_frequency.KmerFrequencyEncoder.KmerFrequencyEncoder

In [341]:
Hpsetting1.encoder.encode(KmerFrequencyEncoder ,  dataset = dataset , params =  Hpsetting1.encoder_params)


TypeError: _prepare_caching_params() missing 1 required positional argument: 'params'

In [ ]:
%python 